In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.DEBUG)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from ionbeam.core.source import Source
from IPython.display import display
from ionbeam.metadata.db import init_db
import sqlalchemy
from datetime import datetime, UTC, timedelta

# init_db(config.globals) 

In [14]:
from copy import deepcopy
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator
from ionbeam.core.singleprocess_pipeline import process_message
from datetime import datetime, timedelta

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["acronet"],
                    download = True,
                    die_on_error = True,
                    reingest_from = datetime.fromisoformat("2021-01-01").replace(tzinfo=UTC),
                    )

config.globals.ingestion_time_constants.query_timespan = dataclasses.replace(
    config.globals.ingestion_time_constants.query_timespan,
    start = datetime.now(UTC) - timedelta(hours = 10))

sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

online = True
source = sources[0]
incoming_message_stream = source.generate()
input_message = next(incoming_message_stream)

possible_actions = [a for a in actions if not isinstance(a, Source)]
message = deepcopy(input_message)

message_history = []
actions_run = set()
while True:
    display(message)
    message_history.append(deepcopy(message))
    matching = list(message.find_matches(config.globals.actions_by_id))
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]
    if action.id in actions_run:
        print(f"Error: loop found for {action} {action.id = }")
        break

    # print(f"current message id {id(message)}")
    message = next(process_message(message, action))
    actions_run.add(action.id)

    
        

DEBUG:ionbeam.core.config_parser.config_parser:Configuration Directory: /Users/math/git/IonBeam_bundle/IonBeam/config
DEBUG:ionbeam.core.config_parser.config_parser:Global config file: /Users/math/git/IonBeam_bundle/IonBeam/config/config.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam_bundle/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.cache_path to /Users/math/git/IonBeam_bundle/data/cache
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam_bundle/IonBeam/config/schema
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_root to /Users/math/git/IonBeam_bundle/data/fdb_root
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.metkit_language_template to /Users/math/git/IonBeam_bundle/IonBeam/config/language.yaml.template


TabularMessage(next=None, metadata=MetaData(source_action_id=UUID('d9417d61-0756-4d8a-a4b5-40f5bae5ba9a'), state=None, source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 18, 4, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 18, 5, 0, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id=None, unstructured={}, columns={'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)')}, internal_id=None, external_id=None, date=None, time=None), history=[], data=                                 external_station_id  RADIOMETRO [W/m^2]  \
datetime                                                                   
2025-01-18 04:00:00+00:00             nino_lamboglia                 NaN   
2025-01-18 04:00:00+00:00                   pluvio_1                 NaN   
2025-01-18 04:00:00+00:00                       faje                 NaN   
2025-01-18 04:00:00+00:00              valle_berlino                 NaN   
2025-01-18 04:00:00+00:00  chiesa_san_bernardo_carpe                 NaN   
...                                              ...                 ...   
2025-01-18 05:00:00+00:00          monte_santa_croce                 NaN   
2025-01-18 05:00:00+00:00            municipio_rezzo                 NaN   
2025-01-18 05:00:00+00:00               località_beo                 NaN   
2025-01-18 05:00:00+00:00  impianti_sportivi_seborga                 NaN   
2025-01-18 05:00:00+00:00      ic_pegli___villa_rosa                 NaN   

                           DIREZIONEVENTO_RAFFICA [Degrees]  \
datetime                                                      
2025-01-18 04:00:00+00:00                               NaN   
2025-01-18 04:00:00+00:00                               NaN   
2025-01-18 04:00:00+00:00                               NaN   
2025-01-18 04:00:00+00:00                               NaN   
2025-01-18 04:00:00+00:00                               NaN   
...                                                     ...   
2025-01-18 05:00:00+00:00                               NaN   
2025-01-18 05:00:00+00:00                               NaN   
2025-01-18 05:00:00+00:00                               NaN   
2025-01-18 05:00:00+00:00                               NaN   
2025-01-18 05:00:00+00:00                               NaN   

                           DIREZIONEVENTO [Degrees]  BAROMETRO [hPa]  \
datetime                                                               
2025-01-18 04:00:00+00:00                 312.30002              NaN   
2025-01-18 04:00:00+00:00                 205.00000              NaN   
2025-01-18 04:00:00+00:00                 231.10000              NaN   
2025-01-18 04:00:00+00:00                       NaN              NaN   
2025-01-18 04:00:00+00:00                 121.80000              NaN   
...                                             ...              ...   
2025-01-18 05:00:00+00:00                  61.50000        949.86000   
2025-01-18 05:00:00+00:00                 134.90001              NaN   
2025-01-18 05:00:00+00:00                 248.40001       1021.11000   
2025-01-18 05:00:00+00:00                 175.90001              NaN   
2025-01-18 05:00:00+00:00                 233.10000       1022.77997   

                           PLUVIOMETRO [mm]  SIGNAL_STRENGTH [CSQ]  \
datetime                                                             
2025-01-18 04:00:00+00:00               0.0                    NaN   
2025-0

That matched with:  ['ComputeColumnMappingsByName(id = ...86e3d)']


TabularMessage(next=[UUID('754e3cfe-d411-4f87-b1ec-29bba51eff0a')], metadata=MetaData(source_action_id=UUID('d9417d61-0756-4d8a-a4b5-40f5bae5ba9a'), state=None, source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 18, 4, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 18, 5, 0, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id=None, unstructured={}, columns={'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'RADIOMETRO [W/m^2]': RawVariable(name='solar_radiation', description=None, unit='W/m^2', type=None, CRS=None, dtype='float64', discard=False, key='RADIOMETRO [W/m^2]', canonical_variable=CanonicalVariable(name='solar_radiation', unit='W/m^2', desc=None), converter=None, metadata={}), 'DIREZIONEVENTO_RAFFICA [Degrees]': RawVariable(name='wind_gust_direction', description=None, unit='°', type=None, CRS=None, dtype='float64', discard=False, key='DIREZIONEVENTO_RAFFICA [Degrees]', canonical_variable=CanonicalVariable(name='wind_gust_direction', unit='°', desc=None), converter=None, metadata={}), 'DIREZIONEVENTO [Degrees]': RawVariable(name='wind_direction_near_surface', description=None, unit='°', type=None, CRS=None, dtype='float64', discard=False, key='DIREZIONEVENTO [Degrees]', canonical_variable=CanonicalVariable(name='wind_direction_near_surface', unit='°', desc='The direction of the wind near the surface in degrees clockwise from north.'), converter=None, metadata={}), 'BAROMETRO [hPa]': RawVariable(name='air_pressure_near_surface', description=None, unit='hPa', type=None, CRS=None, dtype='float64', discard=False, key='BAROMETRO [hPa]', canonical_variable=CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), converter=None, metadata={}), 'PLUVIOMETRO [mm]': RawVariable(name='rainfall', description=None, unit='mm', type=None, CRS=None, dtype='float64', discard=False, key='PLUVIOMETRO [mm]', canonical_variable=CanonicalVariable(name='rainfall', unit='mm', desc=None), converter=None, metadata={}), 'SIGNAL_STRENGTH [CSQ]': RawVariable(name='signal_strength', description=None, unit='CSQ', type=None, CRS=None, dtype='float64', discard=False, key='SIGNAL_STRENGTH [CSQ]', canonical_variable=CanonicalVariable(name='signal_strength', unit='CSQ', desc=None), converter=None, metadata={}), 'BATTERIA [V]': RawVariable(name='battery_level', description=None, unit='V', type=None, CRS=None, dtype='float64', discard=False, key='BATTERIA [V]', canonical_variable=CanonicalVariable(name='battery_level', unit='V', desc='The battery level of the device.'), converter=None, metadata={}), 'ANEMOMETRO [m/s]': RawVariable(name='wind_speed_near_surface', description=None, unit='m/s', type=None, CRS=None, dtype='float64', discard=False, key='ANEMOMETRO [m/s]', canonical_variable=CanonicalVariable(name='wind_speed_near_surface', unit='m/s', desc=None), converter=None, metadata={}), 'ANEMOMETRO_RAFFICA [m/s]': RawVariable(name='wind_gust', description=None, unit='m/s', type=None, CRS=None, dtype='float64', discard=False, key='ANEMOMETRO_RAFFICA [m/s]', canonical_variable=CanonicalVariable(name='wind_gust', unit='m/s', desc=None), converter=None, metadata={}), 'TERMOMETRO [°C]': RawVariable(name='air_temperature_near_surface', description=None, unit='°C', type=None, CRS=None, dtype='float64', discard=False, key='TERMOMETRO [°C]', canonical_variable=CanonicalVariable(name='air_temperature_near_surface', unit='K', desc='The temperature of the air near th

That matched with:  ['SetConstants(id = ...eff0a)']


...

BytesMessage(next=[UUID('1025a4fa-75c7-4623-9758-407d2fd3d19d')], metadata=MetaData(source_action_id=UUID('f8ecedf7-83ab-436a-b7d8-dbc20dddabf4'), state='encoded', source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 18, 4, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 18, 5, 0, tzinfo=datetime.timezone.utc)), encoded_format='parquet', filepath=None, mars_id={'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver': Key(key='expver', value='xxxx', key_spec=expver, reason='Matches', odb_table=None), 'stream': Key(key='stream', value='lwda', key_spec=stream, reason='Matches', odb_table=None), 'aggregation_type': Key(key='aggregation_type', value='by_time', key_spec=aggregation_type=by_time, reason='Matches', odb_table=None), 'platform': Key(key='platform', value='acronet', key_spec=platform, reason='Matches', odb_table=None), 'date': Key(key='date', value='20250118', key_spec=date, reason='Matches', odb_table=None), 'time': Key(key='time', value=datetime.time(4, 0), key_spec=time, reason='Matches', odb_table=None), 'version': Key(key='version', value='1', key_spec=version, reason='Matches', odb_table=None)}, unstructured={}, columns={'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'solar_radiation': CanonicalVariable(name='solar_radiation', unit='W/m^2', desc=None), 'wind_gust_direction': CanonicalVariable(name='wind_gust_direction', unit='°', desc=None), 'wind_direction_near_surface': CanonicalVariable(name='wind_direction_near_surface', unit='°', desc='The direction of the wind near the surface in degrees clockwise from north.'), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'rainfall': CanonicalVariable(name='rainfall', unit='mm', desc=None), 'signal_strength': CanonicalVariable(name='signal_strength', unit='CSQ', desc=None), 'battery_level': CanonicalVariable(name='battery_level', unit='V', desc='The battery level of the device.'), 'wind_speed_near_surface': CanonicalVariable(name='wind_speed_near_surface', unit='m/s', desc=None), 'wind_gust': CanonicalVariable(name='wind_gust', unit='m/s', desc=None), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc='The temperature of the air near the surface.'), 'internal_temperature': CanonicalVariable(name='internal_temperature', unit='K', desc='The internal temperature of the sensor.'), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc='The relative humidity of the air near the surface.'), 'thermometer_max': CanonicalVariable(name='thermometer_max', unit='K', desc=None), 'thermometer_min': CanonicalVariable(name='thermometer_min', unit='K', desc=None), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'chunk_date': CanonicalVariable(name='chunk_date', unit=None, desc='The date of the chunk of data in the format YYYYMMDD'), 'chunk_time': CanonicalVariable(name='chunk_time', unit=None, desc

DEBUG:ionbeam.writers.fdb_writer:Found 0 existing matches for {'class': 'rd', 'expver': 'xxxx', 'stream': 'lwda', 'aggregation_type': 'by_time', 'platform': 'acronet', 'date': '20250118', 'time': '0400', 'version': '1'} in 6.89 ms
DEBUG:ionbeam.writers.fdb_writer:Added new data for {'class': 'rd', 'expver': 'xxxx', 'stream': 'lwda', 'aggregation_type': 'by_time', 'platform': 'acronet', 'date': '20250118', 'time': '0400', 'version': '1'} in 3.15 ms


That matched with:  ['FDBWriter()']


DataMessage(next=None, metadata=MetaData(source_action_id=UUID('1025a4fa-75c7-4623-9758-407d2fd3d19d'), state='written', source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 18, 4, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 18, 5, 0, tzinfo=datetime.timezone.utc)), encoded_format='parquet', filepath=None, mars_id={'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver': Key(key='expver', value='xxxx', key_spec=expver, reason='Matches', odb_table=None), 'stream': Key(key='stream', value='lwda', key_spec=stream, reason='Matches', odb_table=None), 'aggregation_type': Key(key='aggregation_type', value='by_time', key_spec=aggregation_type=by_time, reason='Matches', odb_table=None), 'platform': Key(key='platform', value='acronet', key_spec=platform, reason='Matches', odb_table=None), 'date': Key(key='date', value='20250118', key_spec=date, reason='Matches', odb_table=None), 'time': Key(key='time', value=datetime.time(4, 0), key_spec=time, reason='Matches', odb_table=None), 'version': Key(key='version', value='1', key_spec=version, reason='Matches', odb_table=None)}, unstructured={}, columns={'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'solar_radiation': CanonicalVariable(name='solar_radiation', unit='W/m^2', desc=None), 'wind_gust_direction': CanonicalVariable(name='wind_gust_direction', unit='°', desc=None), 'wind_direction_near_surface': CanonicalVariable(name='wind_direction_near_surface', unit='°', desc='The direction of the wind near the surface in degrees clockwise from north.'), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'rainfall': CanonicalVariable(name='rainfall', unit='mm', desc=None), 'signal_strength': CanonicalVariable(name='signal_strength', unit='CSQ', desc=None), 'battery_level': CanonicalVariable(name='battery_level', unit='V', desc='The battery level of the device.'), 'wind_speed_near_surface': CanonicalVariable(name='wind_speed_near_surface', unit='m/s', desc=None), 'wind_gust': CanonicalVariable(name='wind_gust', unit='m/s', desc=None), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc='The temperature of the air near the surface.'), 'internal_temperature': CanonicalVariable(name='internal_temperature', unit='K', desc='The internal temperature of the sensor.'), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc='The relative humidity of the air near the surface.'), 'thermometer_max': CanonicalVariable(name='thermometer_max', unit='K', desc=None), 'thermometer_min': CanonicalVariable(name='thermometer_min', unit='K', desc=None), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'chunk_date': CanonicalVariable(name='chunk_date', unit=None, desc='The date of the chunk of data in the format YYYYMMDD'), 'chunk_time': CanonicalVariable(name='chunk_time', unit=None, desc='The time of the chunk of data in the form

No more matches


In [7]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

# chains = [[s,] for s in sources]
# for a in processors:
#     if not isinstance(a.match, UUID): 
#         chains.append([str(a.match), a,])
#         continue
#     for c in chains:
#         matches = a.match.int == c[-1].id.int
#         if matches: c.append(a)
            

# for c in chains:
#     print(" --> ".join(str(a) for a in c))

In [8]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


name,None
forward_to_names,None
id,b1c263ef-d032-4c83-b466-8f35191c4635
next,[SetConstants(id = ...4c2f9)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
copy_metadata_to_columns,"{'external_station_id': 'device.id', 'station_name': 'device.name', 'lat': 'device.location.latitude', 'lon': 'device.location.longitude', 'author': 'device.owner.username', 'station_created_at': 'device.created_at'}"
finish_after,None
cache_version,3
use_cache,True
source,smart_citizen_kit
maximum_request_size,"1 day, 0:00:00"


Aggregators
Actions


name,None
forward_to_names,None
id,84568cf8-8ff3-4539-8abe-638b4cf4c2f9
next,[ComputeStationId(id = ...4a4e6)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,b1c263ef-d032-4c83-b466-8f35191c4635
set,"{'platform': 'smart_citizen_kit', 'aggregation_type': 'by_time'}"
name,value
source_action_id,84568cf8-8ff3-4539-8abe-638b4cf4c2f9
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,None
forward_to_names,None
id,91fe6e2f-bcef-48d9-b8a0-be940334a4e6
next,[FormatChecks(id = ...651a1)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,84568cf8-8ff3-4539-8abe-638b4cf4c2f9
name,value
source_action_id,91fe6e2f-bcef-48d9-b8a0-be940334a4e6
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


...

name,None
forward_to_names,None
id,48925e5a-485b-4af7-9037-f5f46d60cc03
next,[ComputeMARSIdentifier(id = ...efa07)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,cf4648e7-9daa-44bb-adc1-5f0eb7e39686
name,value
source_action_id,48925e5a-485b-4af7-9037-f5f46d60cc03
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


name,None
forward_to_names,[final_processing]
id,f47f5718-8744-4766-a5f3-58272a4efa07
next,[IdentityAction(id = ...d3b1d)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,48925e5a-485b-4af7-9037-f5f46d60cc03
lookup,"{'class': 'const.rd', 'expver': 'const.xxxx', 'stream': 'const.lwda', 'aggregation_type': 'data.aggregation_type', 'platform': 'data.platform', 'date': 'data.chunk_date', 'time': 'data.chunk_time', 'version': 'const.1'}"
name,value
source_action_id,f47f5718-8744-4766-a5f3-58272a4efa07
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,final_processing
forward_to_names,None
id,19c149e6-94e9-40b0-acc0-f4e609cd3b1d
next,[ParquetEncoder(id = ...88e30)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,None
name,value
source_action_id,19c149e6-94e9-40b0-acc0-f4e609cd3b1d
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


name,None
forward_to_names,None
id,350d6fc3-252d-4c4b-8822-8e55c4c88e30
next,[FDBWriter()]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,19c149e6-94e9-40b0-acc0-f4e609cd3b1d
name,value
source_action_id,350d6fc3-252d-4c4b-8822-8e55c4c88e30
state,encoded
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,None
forward_to_names,None
id,13ebcc35-653b-47fe-8f7f-e415e813e96b
next,None
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,350d6fc3-252d-4c4b-8822-8e55c4c88e30
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_bundle/data/fdb_root'}]}], 'schema': '/Users/math/git/IonBeam_bundle/IonBeam/config/schema'}"
debug,[]
name,value
source_action_id,13ebcc35-653b-47fe-8f7f-e415e813e96b
state,written


In [9]:
# time_span = TimeSpan.last(days = 50)

# datastreams = list(source.get_data_streams(time_span))
# datastreams

In [10]:
from ionbeam.metadata import db

with Session(config.globals.sql_engine) as db_session:
    for s in db_session.query(db.Station).all():
        if not s._time_span:
            print(s.platform)
            print(s.location)
            print(s.authors)
            
    
    # for ds in datastreams:
    #     existing_chunks = ds.get_chunks(db_session, time_span)
    #     print(list(existing_chunks))
    

In [11]:
# from matplotlib import pyplot as plt
# from datetime import UTC

# show_time_span = TimeSpan.last(days = 100)

# def plot_datastreams(data_streams):

#     n = len(data_streams)
#     f, axes = plt.subplots(n, 1, figsize = (10, 2*n), sharex = True, squeeze = False)
    
#     def plot_timespan(ax, ts, colour, label = None):
#         ax.axvline(ts.start, linestyle = "dotted", color = colour)
#         ax.axvline(ts.end, linestyle = "solid", color = colour)
#         ax.axvspan(ts.start, ts.end, alpha = 0.1, color = colour, label = label)
    
#     with Session(config.globals.sql_engine) as db_session:
#         for ds, ax in zip(data_streams, axes[0]):
#             existing_chunks = ds.get_chunks(db_session, show_time_span)
            
#             for i, chunk in enumerate(existing_chunks):
#                 if not chunk.success:
#                     pprint(chunk)
#                 colour = "red" if not chunk.success else "green"
#                 label = "Failed" if not chunk.success else "Sucess"
#                 plot_timespan(ax, chunk.time_span, colour, label = label)

#                 times = set(datetime.strptime(t, "%Y%m%d%H%M")
#                             for readings in chunk.json
#                             for t in readings["timeline"])
#                 ax.scatter(y = [i%2 for _ in times],  x = list(times), color = "blue")
            
#             gaps = source.gaps_in_database(db_session, ds, show_time_span)
#             for ts in gaps: 
#                 # print(f"gap: {ts.delta()}")
#                 plot_timespan(ax, ts, "blue", label = "Todo")
    
#             ax.axvline(datetime.now(UTC), linestyle = "dotted", color = "k")
#             ax.set(ylabel = ds.key)
#     # ax.legend()

# for source in sources:
#     print(f"Source: {source}")
#     plot_datastreams(list(source.get_data_streams(show_time_span)))

In [12]:
message_history[0].data

sensor_key,external_station_id,bar,bat,co2,eco2,ext_h,ext_t,h,light,noise_dba,pm_avg_1,pm_avg_10,pm_avg_2.5,t,tvoc,station_name,lat,lon,author,station_created_at
datetime,,,,,,,,,,,,,,,,,,,,
2025-01-17 14:08:47+00:00,16774,103.05,100.0,421.0,2388.0,32.37,15.34,41.80,8.0,52.68,NaN,NaN,NaN,12.33,1546.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
2025-01-17 14:08:57+00:00,16517,0.00,99.0,359.0,7833.0,52.89,8.76,62.71,4.0,71.49,NaN,NaN,NaN,5.04,13190.0,I-CHANGE CIMA ARENZANO,44.39930,8.67264,smart_citizen_kit,2023-07-16 21:52:48+00:00
2025-01-17 14:09:12+00:00,16030,103.14,98.0,594.0,1962.0,62.71,11.06,66.47,716.0,68.64,9.0,23.0,19.0,7.69,683.0,I-CHANGE CIMA SAVONA,44.29920,8.45138,smart_citizen_kit,2023-01-30 18:40:26+00:00
2025-01-17 14:09:23+00:00,16643,102.06,99.0,478.0,2088.0,47.83,12.36,54.92,7.0,53.18,NaN,NaN,NaN,9.63,894.0,I-CHANGE CIMA CASTAGNA,44.49617,8.92065,smart_citizen_kit,2023-10-09 06:37:25+00:00
2025-01-17 14:09:47+00:00,16774,103.04,100.0,420.0,2386.0,32.40,15.36,41.57,8.0,60.71,NaN,NaN,NaN,12.31,1537.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 14:58:34+00:00,16643,101.99,99.0,458.0,2082.0,47.87,11.96,54.77,3.0,58.42,NaN,NaN,NaN,9.26,883.0,I-CHANGE CIMA CASTAGNA,44.49617,8.92065,smart_citizen_kit,2023-10-09 06:37:25+00:00
2025-01-17 14:59:06+00:00,16774,102.98,100.0,415.0,2454.0,31.77,15.14,40.57,7.0,53.97,NaN,NaN,NaN,12.03,1678.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
2025-01-17 14:59:07+00:00,16517,0.00,99.0,356.0,8376.0,55.81,8.11,66.02,3.0,71.24,NaN,NaN,NaN,4.48,14345.0,I-CHANGE CIMA ARENZANO,44.39930,8.67264,smart_citizen_kit,2023-07-16 21:52:48+00:00


## Reload and retry failed action

In [13]:
from ionbeam.core.config_parser import reload_action
from IPython.display import display

print(f"Failed action: {action} Definition: {action.definition_path}")

config, action = reload_action(config, action)

message = next(action.process(message_history[-1]))

display(action)
display(message)

DEBUG:ionbeam.core.config_parser.config_parser:Reloading FDBWriter() from /Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Configuration Directory: /Users/math/git/IonBeam_bundle/IonBeam/config
DEBUG:ionbeam.core.config_parser.config_parser:Global config file: /Users/math/git/IonBeam_bundle/IonBeam/config/config.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam_bundle/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.cache_path to /Users/math/git/IonBeam_bundle/data/cache
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam_bundle/IonBeam/config/schema
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_root to /Users/math/git/IonBeam_bundle/data/fdb_root


Failed action: FDBWriter() Definition: /Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml


DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.metkit_language_template to /Users/math/git/IonBeam_bundle/IonBeam/config/language.yaml.template
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.secrets_file to /Users/math/git/IonBeam_bundle/IonBeam/config/secrets.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Checked repository source state: CodeSourceInfo(repo_status='Dirty', git_hash='63249de700a5071624275807640be737b969256d')
DEBUG:ionbeam.writers.fdb_writer:FDBWriter using fdb5 shared library from /Users/math/micromamba/envs/ionbeam/lib/libfdb5.dylib
DEBUG:ionbeam.writers.fdb_writer:Installing metkit overlays
DEBUG:ionbeam.writers.fdb_writer:language.yaml backup exists at /Users/math/micromamba/envs/ionbeam/share/metkit/language.yaml.original
DEBUG:ionbeam.writers.fdb_writer:marsrequest.yaml backup exists at /Users/math/micromamba/envs/ionbeam/share/metkit/odb/marsrequest.yaml.original
DEBUG:ionbeam.writers.fdb_writer:Writing langua

AttributeError: 'DataMessage' object has no attribute 'data_bytes'

In [ ]:
m = message_history[-1]
m.metadata.unstructured

In [ ]:
import pyodc as pyodc
from io import BytesIO

f = BytesIO(message.bytes)
df = pyodc.read_odb(f, single = True)
df

In [ ]:
from ionbeam.core.converters import unit_conversions

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100)})

In [ ]:
converter = unit_conversions["°C -> K"]


%timeit df.apply({"A" : converter}, raw = True, by_row = False)

In [ ]:
converter = lambda t: t + 273.15
converter2 = lambda t: t * 1000

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100), "C" : np.arange(100)})

df = df.apply({"A" : converter, "B" : converter2}, raw = True, by_row = False)

df

In [ ]:
converter = lambda 